In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# import pymc3 as pm
import math
import pandas as pd
import arviz as az
import pickle

In [ ]:
import pickle

In [ ]:
T_home = np.arange(100)
N_home = np.arange(950)
TN_home = np.empty((100, 950))
R_home = np.empty((100, 950))
X_home = np.empty((100, 9, 950))
t_home = TN_home.copy()

In [ ]:
with pm.Model() as home_model:
    ha = pm.Normal("ha", mu=0, sigma=math.sqrt(2))
    # at = pm.Normal("at", mu=0, sigma=math.sqrt(2), shape=(100,8))
    at = pm.GaussianRandomWalk("at", mu=0, sigma=math.sqrt(2), shape=(100, 9))
    beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

    data = pm.Data("data", X_home)
    theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta + ha)
    # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)

    like = pm.Poisson("like", (100 - t_home) * theta, observed=R_home)

pm.model_to_graphviz(home_model)

In [ ]:
home_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/all_games_home.csv")

In [ ]:
# create test data
X_test = np.zeros((100, 9, 950))
## create X
for i, row in home_data.iterrows():
    if int(i / 100) >= 950:
        for x in range(len(row)):
            if x == 9:
                continue
            X_test[int(row[0])][x][int(i / 100 - 950)] = row[x]

# X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
X_test.shape

In [ ]:
trace = az.from_netcdf(
    "/home/morten/Develop/Live-Win-Prob/models/random_walk_home_only_theta.nc"
)

In [ ]:
with home_model:
    pm.set_data({"data": X_test}, model=home_model)
    ppc_test = pm.sample_posterior_predictive(trace, model=home_model, samples=1000)

In [ ]:
ppc_test

In [ ]:
with open("home_random_walk.pkl", "wb") as f:
    pickle.dump(ppc_test, f)
f.close()

In [ ]:
ppc_test["like"].shape

In [ ]:
ppc_only_actual_params = ppc_test["like"][:, :, :53]

In [ ]:
ppc_only_actual_params.shape

In [ ]:
ppc_only_params_game_one_home = ppc_only_actual_params[:, :, 0]
ppc_only_params_game_one_home.shape

In [ ]:
ppc_only_params_game_one_home[:, 0].shape

In [ ]:
# create plot

In [ ]:
T_away = np.arange(100)
N_away = np.arange(950)
TN_away = np.empty((100, 950))
R_away = np.empty((100, 950))
X_away = np.empty((100, 9, 950))
t_away = TN_away.copy()

In [ ]:
with pm.Model() as away_model:
    # at = pm.Normal("at", mu=0, sigma=math.sqrt(2), shape=(100,9))
    at = pm.GaussianRandomWalk("at", mu=0, sigma=math.sqrt(2), shape=(100, 9))
    beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

    data = pm.Data("data", X_away)
    theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta)
    # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)

    like = pm.Poisson("like", (100 - t_away) * theta, observed=R_away)

pm.model_to_graphviz(away_model)

In [ ]:
away_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/all_games_away.csv")

In [ ]:
# create test data
X_test_away = np.zeros((100, 9, 950))
## create X
for i, row in away_data.iterrows():
    if int(i / 100) >= 950:
        for x in range(len(row)):
            if x == 9:
                continue
            X_test_away[int(row[0])][x][int(i / 100 - 950)] = row[x]

# X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
X_test_away.shape

In [ ]:
trace_away = az.from_netcdf(
    "/home/morten/Develop/Live-Win-Prob/models/random_walk_only_theta.nc"
)

In [ ]:
with away_model:
    pm.set_data({"data": X_test_away}, model=away_model)
    ppc_test_away = pm.sample_posterior_predictive(
        trace_away, model=away_model, samples=1000
    )

In [ ]:
ppc_test_away

In [ ]:
with open("away_random_walk.pkl", "wb") as f:
    pickle.dump(ppc_test_away, f)
f.close()

In [ ]:
ppc_only_actual_params_away = ppc_test_away["like"][:, :, :53]

In [ ]:
plt.hist(ppc_only_actual_params_away[:, 20, 0])

In [ ]:
## create plots

In [ ]:
with open("/home/morten/Develop/Live-Win-Prob/dumps/home_ppc.pkl", "rb") as f:
    home_ppc = pickle.load(f)
with open("/home/morten/Develop/Live-Win-Prob/dumps/away_ppc.pkl", "rb") as f:
    away_ppc = pickle.load(f)

In [ ]:
R_test_home = np.zeros((100, 950))
## create R
for i, row in home_data.iterrows():
    if int(i / 100) >= 950:
        R_test_home[int(row[0])][int(i / 100 - 950)] = int(row[9])
R_test_away = np.zeros((100, 950))
## create R
for i, row in away_data.iterrows():
    if int(i / 100) >= 950:
        R_test_away[int(row[0])][int(i / 100 - 950)] = int(row[9])

In [ ]:
ppc_only_actual_params_away = away_ppc["like"][:, :, :53]
ppc_only_actual_params = home_ppc["like"][:, :, :53]

In [ ]:
game_num = 0

In [ ]:
ppc_only_actual_params_away_game = ppc_only_actual_params_away[:, :, game_num]
ppc_only_actual_params_away_game.shape
ppc_only_actual_params_home_game = ppc_only_actual_params[:, :, game_num]
ppc_only_actual_params_home_game.shape

In [ ]:
game_probs = np.zeros((100, 3))
goal_home_idx = []
goal_away_idx = []
for i in range(100):
    hist_home = np.histogram(
        ppc_only_actual_params_home_game[:, i],
        bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        density=True,
    )
    hist_away = np.histogram(
        ppc_only_actual_params_away_game[:, i],
        bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        density=True,
    )
    goals_home = R_test_home[0][game_num] - R_test_home[i][game_num]
    goals_away = R_test_away[0][game_num] - R_test_away[i][game_num]
    if i > 0:
        if R_test_home[i][game_num] != R_test_home[i - 1][game_num]:
            goal_home_idx.append(i)
        if R_test_away[i][game_num] != R_test_away[i - 1][game_num]:
            goal_away_idx.append(i)
    goal_matrix = np.zeros((9, 9))
    home_prob = 0
    away_prob = 0
    draw_prob = 0
    for h in range(9):
        for a in range(9):
            goal_matrix[h][a] = hist_home[0][h] * hist_away[0][a]
            home = goals_home + h
            away = goals_away + a
            if home > away:
                home_prob += hist_home[0][h] * hist_away[0][a]
            elif away > home:
                away_prob += hist_home[0][h] * hist_away[0][a]
            elif home == away:
                draw_prob += hist_home[0][h] * hist_away[0][a]

    game_probs[i][0] = home_prob
    game_probs[i][1] = draw_prob
    game_probs[i][2] = away_prob

In [ ]:
xx = np.arange(100)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(game_probs[:, 0], color="green")
ax.plot(game_probs[:, 1], color="black")
ax.plot(game_probs[:, 2], color="red")
for goal in goal_away_idx:
    ax.axvline(goal, color="red", linestyle="--")
for goal in goal_home_idx:
    ax.axvline(goal, color="green", linestyle="--")

ax.fill_between(
    xx, y1=game_probs[:, 0], y2=0, alpha=0.5, color="green", linewidth=2, label="Home"
)
ax.fill_between(
    xx, y1=game_probs[:, 1], y2=0, alpha=0.5, color="grey", linewidth=2, label="Draw"
)
ax.fill_between(
    xx, y1=game_probs[:, 2], y2=0, alpha=0.5, color="red", linewidth=2, label="Away"
)

y_ticks = [0.25, 0.5, 0.75, 1]
y_labels = ["25%", "50%", "75%", "100%"]
x_ticks = [0, 16, 33, 50, 66, 83, 100]
x_labels = ["0", "15", "30", "45", "60", "75", "90"]
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_labels)
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_labels)
ax.set_ylabel("Win Percentage")
ax.set_xlabel("Minutes")
ax.grid(axis="y", color="black")

# ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
#           ncol=3, fancybox=True, shadow=False)
ax.legend(loc="best", fontsize=8)

fig.show()

In [ ]:
len(home_ppc["like"][pred][time][game])

In [ ]:
for i in range(53):
    game_num = i

    game_probs = np.zeros((100, 3))
    goal_home_idx = []
    goal_away_idx = []
    for i in range(100):
        hist_home = np.histogram(
            home_ppc["like"][:, i, game_num],
            bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            density=True,
        )
        hist_away = np.histogram(
            away_ppc["like"][:, i, game_num],
            bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            density=True,
        )
        goals_home = R_test_home[0][game_num] - R_test_home[i][game_num]
        goals_away = R_test_away[0][game_num] - R_test_away[i][game_num]
        if i > 0:
            if R_test_home[i][game_num] != R_test_home[i - 1][game_num]:
                goal_home_idx.append(i)
            if R_test_away[i][game_num] != R_test_away[i - 1][game_num]:
                goal_away_idx.append(i)
        goal_matrix = np.zeros((9, 9))
        home_prob = 0
        away_prob = 0
        draw_prob = 0
        for h in range(9):
            for a in range(9):
                goal_matrix[h][a] = hist_home[0][h] * hist_away[0][a]
                home = goals_home + h
                away = goals_away + a
                if home > away:
                    home_prob += hist_home[0][h] * hist_away[0][a]
                elif away > home:
                    away_prob += hist_home[0][h] * hist_away[0][a]
                elif home == away:
                    draw_prob += hist_home[0][h] * hist_away[0][a]

        game_probs[i][0] = home_prob
        game_probs[i][1] = draw_prob
        game_probs[i][2] = away_prob

    fig = plt.figure(figsize=[8.2, 4.8])
    ax = fig.add_subplot(111)
    x = np.arange(100)
    ax.bar(x, height=game_probs[:, 0], width=1, color="springgreen", label="Home")
    ax.bar(
        x,
        height=game_probs[:, 1],
        width=1,
        color="lightsteelblue",
        bottom=game_probs[:, 0],
        label="Draw",
    )
    ax.bar(
        x,
        height=game_probs[:, 2],
        width=1,
        color="lightcoral",
        bottom=(game_probs[:, 0] + game_probs[:, 1]),
        label="Away",
    )

    goal_pos = []
    # minute, team, goal_number
    for idx, goal in enumerate(goal_away_idx):
        ax.axvline(goal, ymax=1, color="black", linestyle="--")
        goal_pos.append((goal, 1, idx + 1))
    for idx, goal in enumerate(goal_home_idx):
        ax.axvline(goal, ymax=1, color="black", linestyle="--")
        goal_pos.append((goal, 0, idx + 1))

    goal_pos.sort(key=lambda tup: tup[0])  # sorts in place
    home_goals_so_far = 0
    away_goals_so_far = 0
    for x in goal_pos:
        if x[1] == 0:
            home_goals_so_far += 1
        else:
            away_goals_so_far += 1

        ax.text(
            x[0],
            1.02,
            str(home_goals_so_far) + ":" + str(away_goals_so_far),
            ha="center",
        )

    y_ticks = [0.25, 0.5, 0.75, 1]
    y_labels = ["25%", "50%", "75%", "100%"]
    x_ticks = [0, 16, 33, 50, 66, 83, 99]
    x_labels = ["0", "15", "30", "45", "60", "75", "90"]
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_labels)
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_labels)
    ax.set_ylabel("Win Percentage")
    ax.set_xlabel("Minutes", loc="left")
    ax.grid(axis="y", color="black")
    ax.legend(
        loc="upper center",
        bbox_to_anchor=(0.5, -0.05),
        fancybox=True,
        shadow=True,
        ncol=5,
    )
    # ax.legend(loc="best", fontsize=8)
    ax.margins(x=0, y=0)

    fig.savefig(f"/home/morten/Develop/Live-Win-Prob/media/out/Oldgame{game_num}.jpg")

    fig.clf()

In [ ]:
B_game_probs = np.zeros((53, 100, 3))
game_pred = np.zeros((53, 100))
acc_pred = np.zeros((53, 100))

In [ ]:
for g in range(53):
    game_num = g
    ppc_only_actual_params_away_game = ppc_only_actual_params_away[:, :, game_num]
    ppc_only_actual_params_away_game.shape
    ppc_only_actual_params_home_game = ppc_only_actual_params[:, :, game_num]
    ppc_only_actual_params_home_game.shape

    game_probs = np.zeros((100, 3))
    goal_home_idx = []
    goal_away_idx = []
    for i in range(100):
        hist_home = np.histogram(
            ppc_only_actual_params_home_game[:, i],
            bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            density=True,
        )
        hist_away = np.histogram(
            ppc_only_actual_params_away_game[:, i],
            bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            density=True,
        )
        goals_home = R_test_home[0][game_num] - R_test_home[i][game_num]
        goals_away = R_test_away[0][game_num] - R_test_away[i][game_num]
        if i > 0:
            if R_test_home[i][game_num] != R_test_home[i - 1][game_num]:
                goal_home_idx.append(i)
            if R_test_away[i][game_num] != R_test_away[i - 1][game_num]:
                goal_away_idx.append(i)
        goal_matrix = np.zeros((9, 9))
        home_prob = 0
        away_prob = 0
        draw_prob = 0
        for h in range(9):
            for a in range(9):
                goal_matrix[h][a] = hist_home[0][h] * hist_away[0][a]
                home = goals_home + h
                away = goals_away + a
                if home > away:
                    home_prob += hist_home[0][h] * hist_away[0][a]
                elif away > home:
                    away_prob += hist_home[0][h] * hist_away[0][a]
                elif home == away:
                    draw_prob += hist_home[0][h] * hist_away[0][a]

        B_game_probs[g][i][0] = home_prob
        B_game_probs[g][i][1] = draw_prob
        B_game_probs[g][i][2] = away_prob
        if home_prob >= draw_prob and home_prob >= away_prob:
            game_pred[g][i] = 0
            acc_pred[g][i] = home_prob
        elif away_prob >= draw_prob and away_prob >= home_prob:
            game_pred[g][i] = 2
            acc_pred[g][i] = away_prob
        elif draw_prob >= away_prob and draw_prob >= home_prob:
            game_pred[g][i] = 1
            acc_pred[g][i] = away_prob

In [ ]:
act_res = np.zeros((53))
for g in range(53):
    goals_diff = R_test_home[0][g] - R_test_away[0][g]
    if goals_diff < 0:
        act_res[g] = 2
    elif goals_diff == 0:
        act_res[g] = 1

In [ ]:
act_res

In [ ]:
trace = az.from_netcdf(
    "/home/morten/Develop/Live-Win-Prob/models/away_trace_advi_with_t.nc"
)

In [ ]:
trace_df = trace.to_dict()

In [ ]:
trace_df["posterior"]["at"][0][0][:, 0].shape

In [ ]:
trace_df["posterior"]["at"][0].shape

In [ ]:
trace_df["posterior"]["at"][0][:, 97, 3].shape

In [ ]:
trace_viz = trace_df["posterior"]["at"][0]

In [ ]:
trace_viz = np.mean(trace_viz, axis=0)
trace_viz.shape

In [ ]:
name = 0
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = trace_df["posterior"]["at"][0][:, i, name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = 1
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = trace_df["posterior"]["at"][0][:, i, name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = 2
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = trace_df["posterior"]["at"][0][:, i, name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = 3
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = trace_df["posterior"]["at"][0][:, i, name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = 4
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = trace_df["posterior"]["at"][0][:, i, name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = 5
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = trace_df["posterior"]["at"][0][:, i, name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = 6
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = trace_df["posterior"]["at"][0][:, i, name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = 7
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = trace_df["posterior"]["at"][0][:, i, name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = 8
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = trace_df["posterior"]["at"][0][:, i, name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
plt.plot(trace_viz[:, 0])

In [ ]:
plt.plot(trace_viz[:, 1])

In [ ]:
plt.plot(trace_viz[:, 2])

In [ ]:
plt.plot(trace_viz[:, 3])

In [ ]:
plt.plot(trace_viz[:, 4])

In [ ]:
plt.plot(trace_viz[:, 5])

In [ ]:
plt.plot(trace_viz[:, 6])

In [ ]:
plt.plot(trace_viz[:, 7])

In [ ]:
plt.plot(trace_viz[:, 8])

In [ ]:
## ece
## first half

In [ ]:
# create B:
B_fh = B_game_probs[:, :50, :]

In [ ]:
def acc_ece(Bm, pred, label):
    pred = np.array(pred)
    mul = 1 / pred.size
    sum_ece = 0
    for x in range(len(label)):
        for i in range(len(pred[x])):
            if pred[x][i] == label[x]:
                sum_ece += 1
    return mul * sum_ece

In [ ]:
def conf_ece(Bm, prob):
    prob = np.array(prob)
    mul = mul = 1 / prob.size
    mul2 = np.sum(prob)
    return mul * mul2

In [ ]:
M = 5
Bm = []
game_pred_Bm = []
acc_pred_Bm = []
# label_Bm = []
step_size = len(B_fh[0]) / M
for i in range(M):
    Bm.append(B_fh[:, int((i * step_size)) : int(((i + 1) * step_size)), :])
    game_pred_Bm.append(game_pred[:, int((i * step_size)) : int(((i + 1) * step_size))])
    acc_pred_Bm.append(acc_pred[:, int((i * step_size)) : int(((i + 1) * step_size))])
#    label_Bm.append(act_res[int((i*step_size)):int(((i+1)*step_size))])

In [ ]:
N = 53
m = 1
ece = 0
while m - 1 <= M - 1:
    mul = len(Bm[m - 1]) / N
    acc = acc_ece(Bm[m - 1], game_pred_Bm[m - 1], act_res)

    conf = conf_ece(Bm[m - 1], acc_pred_Bm[m - 1])
    ece += (1 / 5) * mul * np.abs(acc - conf)
    m += 1

print(ece)